In [1]:
# Import necessary libraries
import os
import sys
import json
import numpy as np
import pandas as pd
from datetime import datetime
import tensorflow as tf
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Add parent directory to Python path
sys.path.append('..')

# Import our modules
from src.data import DataProcessor
from src.models import ModelWrapper
from src.visualization import plot_training_history, plot_power_predictions
from src.utils import set_seeds

2024-11-04 15:51:02.966281: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-04 15:51:02.974905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-04 15:51:02.983231: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-04 15:51:02.985642: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-04 15:51:02.992711: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Set random seed for reproducibility
set_seeds(42)

2024-11-04 15:51:07,850 - INFO - Random seeds set to 42


In [3]:
# Create results directory
results_dir = os.path.join('..', 'results', datetime.now().strftime('%Y%m%d_%H%M%S'))
os.makedirs(results_dir, exist_ok=True)

In [5]:
# Step 1: Load and Process Data
file_path = '../data/raw/Train.csv'
target_column = 'Power'

# Initialize Data Processor and prepare data
data_processor = DataProcessor(file_path, target_column)
train_data, val_data, test_data = data_processor.prepare_data()

# Print data shapes for verification
print("\nData Shapes:")
print(f"Training data: {train_data.element_spec[0].shape}")
print(f"Validation data: {val_data.element_spec[0].shape}")
print(f"Test data: {test_data.element_spec[0].shape}")


Data Ranges (scaled):
Training   - X: (98101, 15, 11), y: min=0.0000, max=1.0000
Validation - X: (28029, 15, 11), y: min=0.0000, max=0.9971
Test      - X: (14015, 15, 11), y: min=0.0000, max=0.9969

Temporal Split Check:
Training period: 2013-01-02 00:00:00 to 2015-10-20 21:00:00
Validation period: 2015-10-20 21:00:00 to 2016-08-07 20:00:00

Data Shapes:
Training data: (None, 15, 11)
Validation data: (None, 15, 11)
Test data: (None, 15, 11)


In [6]:
# Step 2: Initialize Model
model_wrapper = ModelWrapper(
    input_shape=(train_data.element_spec[0].shape[1], train_data.element_spec[0].shape[2])
)

# Display model architecture
model_wrapper.model.summary()

/home/dkat/courses/computational-intelligence/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 15, 64)         │        19,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 15, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,905 (124.63 KB)

 Trainable params: 31,905 (124.63 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Step 3: Train Model
print("\nTraining model...")
history = model_wrapper.fit(train_data, val_data)


Training model...
Epoch 1/50


2024-11-04 15:51:23.106859: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


3066/3066 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.1331 - mae: 0.1889 - val_loss: 0.0461 - val_mae: 0.1745 - learning_rate: 0.0010
Epoch 2/50
3066/3066 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0438 - mae: 0.1652 - val_loss: 0.0426 - val_mae: 0.1639 - learning_rate: 0.0010
Epoch 3/50
3066/3066 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0413 - mae: 0.1602 - val_loss: 0.0410 - val_mae: 0.1593 - learning_rate: 0.0010
Epoch 4/50
3066/3066 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0404 - mae: 0.1585 - val_loss: 0.0402 - val_mae: 0.1559 - learning_rate: 0.0010
Epoch 5/50
3066/3066 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0395 - mae: 0.1561 - val_loss: 0.0397 - val_mae: 0.1547 - learning_rate: 0.0010
Epoch 6/50
3066/3066 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0385 - mae: 0.1537 - val_loss: 0.0390 - val_mae: 0.1535 - learning_rate: 0.0010
Epoch 7/50
3066/3066 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0377 - mae: 0.1514 - val_loss: 0.0387 - val_mae: 0.1530 - learning_rate: 0.0

In [8]:
# Step 4: Visualize Training History
print("\nPlotting training history...")
plot_training_history(
    history.history, 
    save_path=os.path.join(results_dir, 'training_history.png')
)

2024-11-04 16:07:17,766 - WARNING - Seaborn style not found. Using default style.
2024-11-04 16:07:17,767 - INFO - Plot style set
2024-11-04 16:07:17,832 - INFO - Training history plot saved to ../results/20241104_155109/training_history.png



Plotting training history...


In [9]:
# Step 5: Evaluate on Test Set
print("\nEvaluating model on test set...")
test_loss = model_wrapper.model.evaluate(test_data, verbose=1)
print(f"Test Loss: {test_loss}")


Evaluating model on test set...
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0255 - mae: 0.1169
Test Loss: [0.03417302295565605, 0.1351407766342163]


In [12]:
# Step 6: Generate Predictions
print("\nGenerating predictions...")
predictions = model_wrapper.model.predict(test_data)

# Extract actual values from test dataset
test_actual = np.concatenate([y for x, y in test_data], axis=0)

# Get corresponding timestamps (using the last 10% of the data for test set)
total_samples = len(data_processor.data)
test_size = int(total_samples * 0.1)  # 10% test split
# Adjust timestamps to match predictions length
test_timestamps = data_processor.data.iloc[-len(test_actual):]['Time'].values

# Print shapes for debugging
print("\nShapes:")
print(f"predictions shape: {predictions.shape}")
print(f"test_actual shape: {test_actual.shape}")
print(f"timestamps shape: {test_timestamps.shape}")

# Inverse transform predictions and actual values for proper comparison
predictions_unscaled = data_processor.target_scaler.inverse_transform(predictions)
actual_unscaled = data_processor.target_scaler.inverse_transform(test_actual.reshape(-1, 1))

print(f"predictions_unscaled shape: {predictions_unscaled.shape}")
print(f"actual_unscaled shape: {actual_unscaled.shape}")

# Step 7: Visualize Predictions
print("\nPlotting predictions...")
plot_power_predictions(
    actual=actual_unscaled.flatten(),
    predicted=predictions_unscaled.flatten(),
    timestamps=test_timestamps,
    save_path=os.path.join(results_dir, 'predictions.png')
)


Generating predictions...
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


2024-11-04 16:09:23.439722: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-11-04 16:09:23,442 - WARNING - Seaborn style not found. Using default style.
2024-11-04 16:09:23,443 - INFO - Plot style set



Shapes:
predictions shape: (14015, 1)
test_actual shape: (14015,)
timestamps shape: (14015,)
predictions_unscaled shape: (14015, 1)
actual_unscaled shape: (14015, 1)

Plotting predictions...


2024-11-04 16:09:24,245 - INFO - Power predictions plot saved to ../results/20241104_155109/predictions.png


In [13]:
# Step 7: Visualize Predictions
print("\nPlotting predictions...")
plot_power_predictions(
    actual=actual_unscaled.flatten(),
    predicted=predictions_unscaled.flatten(),
    timestamps=test_timestamps,
    save_path=os.path.join(results_dir, 'predictions.png')
)

2024-11-04 16:09:27,862 - WARNING - Seaborn style not found. Using default style.
2024-11-04 16:09:27,863 - INFO - Plot style set



Plotting predictions...


2024-11-04 16:09:28,640 - INFO - Power predictions plot saved to ../results/20241104_155109/predictions.png


In [17]:
# Step 8: Save Model and Results
print("\nSaving model and results...")

# Save model weights
model_weights_path = os.path.join(results_dir, 'model.weights.h5')
model_wrapper.model.save_weights(model_weights_path)

# Save model architecture
model_json = model_wrapper.model.to_json()
with open(os.path.join(results_dir, 'model_architecture.json'), 'w') as f:
    f.write(model_json)

# Save training history
with open(os.path.join(results_dir, 'training_history.json'), 'w') as f:
    json.dump(history.history, f)

# Calculate comprehensive metrics
mse = mean_squared_error(actual_unscaled, predictions_unscaled)
rmse = np.sqrt(mse)
r2 = r2_score(actual_unscaled, predictions_unscaled)
mae = np.mean(np.abs(predictions_unscaled - actual_unscaled))

# Get model evaluation metrics
test_metrics = model_wrapper.model.evaluate(test_data, verbose=0)
test_loss = test_metrics[0] if isinstance(test_metrics, list) else test_metrics

# Get configuration details
model_config = {
    'look_back': 15,  # from data processor
    'lstm_units': [64, 32],  # from model architecture
    'dropout_rate': 0.2,
    'learning_rate': 0.001,
    'batch_size': 32,
    'early_stopping_patience': 5,
    'reduce_lr_factor': 0.5,
    'reduce_lr_patience': 3,
    'min_lr': 1e-5
}

# Calculate dataset sizes
train_size = sum(1 for _ in train_data)
val_size = sum(1 for _ in val_data)
test_size = sum(1 for _ in test_data)

# Save all results
results = {
    'model_config': model_config,
    'test_metrics': {
        'loss': float(test_loss),
        'mse': float(mse),
        'rmse': float(rmse),
        'r2': float(r2),
        'mae': float(mae)
    },
    'data_info': {
        'train_size': train_size,
        'val_size': val_size,
        'test_size': test_size,
        'features': list(data_processor.data.columns),
        'target': target_column
    },
    'training_info': {
        'final_epoch': len(history.history['loss']),
        'best_val_loss': float(min(history.history['val_loss'])),
        'final_val_loss': float(history.history['val_loss'][-1])
    },
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

# Save all results
with open(os.path.join(results_dir, 'results.json'), 'w') as f:
    json.dump(results, f, indent=4)

# Print final metrics
print("\nFinal Test Metrics:")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")
print(f"MAE: {mae:.4f}")

print(f"\nResults saved in: {results_dir}")

# Save scaled data ranges for future reference
data_ranges = {
    'input_features': {
        'min': float(data_processor.scaler.data_min_[0]),
        'max': float(data_processor.scaler.data_max_[0])
    },
    'target': {
        'min': float(data_processor.target_scaler.data_min_[0]),
        'max': float(data_processor.target_scaler.data_max_[0])
    }
}

with open(os.path.join(results_dir, 'data_ranges.json'), 'w') as f:
    json.dump(data_ranges, f, indent=4)


Saving model and results...

Final Test Metrics:
MSE: 0.0332
RMSE: 0.1822
R²: 0.4635
MAE: 0.1336

Results saved in: ../results/20241104_155109


2024-11-04 16:16:18.282766: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
